FACE RECOGNITION WITH DEPTH ESTIMATION (PERSON TO CAMERA LENGHT)

In [1]:
# env with installed cv2,numpy,cvzone and mediapipe (here: projektSIZE)
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector
import numpy as np
import os 

In [6]:
# used trained recognizer from previous script(face_recognition_1) and added depth estimator

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('C:/Skidanje/projektii/projekt_deepface/tomi/trainer10.yml')
cascadePath = "C:/Users/pc/anaconda3/envs/deepface/Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath)

detector=FaceMeshDetector(maxFaces=3)

font = cv2.FONT_HERSHEY_SIMPLEX

#iniciate id counter
id = 0

# names related to ids: example ==> tomi: id=1,  etc
names = ['None','tomi','ivana','zlatko'] 

# Initialize and start realtime video capture
cam = cv2.VideoCapture(0)
cam.set(3, 640) # set video widht
cam.set(4, 480) # set video height

# Define min window size to be recognized as a face
minW = 0.1*cam.get(3)
minH = 0.1*cam.get(4)
#i=0   # counter for saving frames
while(cam.isOpened()):

    ret, img =cam.read()
    
    img,face=detector.findFaceMesh(img,draw=False)   # part 1 that gives depth(distance to cam)
    for face1 in face:
          # for every face detected make points
        pointLeft = face1[145]  # left eye
        pointRight = face1[374]  # right eye

        # Drawing
        #cv2.line(img, pointLeft, pointRight, (0, 200, 0), 3)
        #cv2.circle(img, pointLeft, 5, (255, 0, 255), cv2.FILLED)
        #cv2.circle(img, pointRight, 5, (255, 0, 255), cv2.FILLED)
        w, _ = detector.findDistance(pointLeft, pointRight)  # find distance between points, width in pixels

        # Finding focal lenght
        W = 6.3  # default for men, width in cm
        #d=50   # you must be 50 cm from camera
        #f=(w*d)/W
        # print(f)

        # Finding object(face) distance in cm
        f=840  #cca
        d = (W*f)/w
        #print(d)

        cvzone.putTextRect(img, f'Depth: {int(d)}cm',
                           (face1[10][0] - 100, face1[10][1] - 50),
                           scale=2)
    
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           # part 2 that gives face recognition

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = recognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id = names[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img)

    if ret == False:
        break
    #cv2.imwrite('Frame'+str(i)+'.jpg', img)       #  save frames until camera is on
    #i += 1

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break


cam.release()
cv2.destroyAllWindows()